In [1]:
# Imports
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import csv
import requests
import re

In [2]:
def get_movies():
    pages = range(1,11)
    
    # Lists of all movie values we intend to include in the dataframe
    movie_titles = []
    movie_ids =  []
    release_years = []
    ratings = []
    summaries = []
    directors = []
    actors = []
    certificates = []
    runtimes = []
    genres = []
    lifetime_grosses = []
    publishers = []
    reviews = []
    review_score = []
    review_users = []
    imdb_page = []
    
    # Count for used for scraping duration
    count = 0
    
    # Headers
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'}
    
    for page in pages:
        url = f"https://www.imdb.com/list/ls098063263/?sort=list_order,asc&st_dt=&mode=detail&page={page}"
        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")


        movie_divs = soup.find_all("div", {"class": "lister-item mode-detail"})


        for div in movie_divs:
            #Header - Title, ID and Release year
            header = div.find("h3", {"class": "lister-item-header"})

            # Parser for the header html
            parse_header = BeautifulSoup(str(header), 'html.parser')

            # Get titles
            title = parse_header.find('a').text
            movie_titles.append(title)

            # Get ID
            title_tag = str(parse_header.find('a'))


            movie_id = title_tag.split('/title/')[1].split('/')[0]

            movie_ids.append(movie_id)

            # Get release years
            year = parse_header.find('span', {'class': 'lister-item-year text-muted unbold'}).text

            # Remove (I) and parenthesis
            year = year.replace("(", "").replace(")", "").strip()

            release_years.append(year)

            p1 = div.find_all("p", {"class": "text-muted text-small"})[0]
            p2 = div.find_all("p", {"class": "text-muted text-small"})[1]
            p3 = div.find_all("p", {"class": "text-muted text-small"})[2]

            # Remove all tags
            cleaned_string_p1 = re.sub('<[^>]*>|\\|', '', str(p1))
            cleaned_string_p2 = re.sub('<[^>]*>|\\|', '', str(p2))

            # Rating
            rating_div = div.find("div", {"class": "ipl-rating-star small"})

            # Parser for the rating html
            parse_rating_div = BeautifulSoup(str(rating_div), 'html.parser')

            #Get rating
            rating = parse_rating_div.find('span', {'class': 'ipl-rating-star__rating'}).text
            ratings.append(rating)

            # Summary
            summary = div.find_all("p", {"class": ""})[0]
            summary = re.sub('<[^>]*>|\\|', '', str(summary)).replace('\n', '')
            summaries.append(summary)

            # Lifetime Gross
            lifetime_gross = div.find_all("p", {"class": ""})[1].find_all('b')
            lifetime_gross = re.sub('<[^>]*>|\\|', '', str(lifetime_gross)).replace('\n', '').replace('[', '').replace(']', '').replace('$', '').replace(',', '')

            lifetime_grosses.append(int(lifetime_gross))

            # p1 - Certificates, Runtime and Generes
            cleaned_string_p1 = cleaned_string_p1.replace('\n', ';')
            fields = cleaned_string_p1.split(';')
            cleaned_fields = [field.strip() for field in fields if field.strip() != '']

            certificate = ""
            runtime = ""
            genre = ""

            if(len(cleaned_fields) == 2):
                certificate = 'Not Rated'
                runtime = cleaned_fields[0]
                genre = cleaned_fields[1]
            else:
                certificate = cleaned_fields[0]
                runtime = cleaned_fields[1]
                genre = cleaned_fields[2]

            p1_dict = {
            "certificate": certificate,
            "runtime": runtime,
            "genre": genre    
            }

            # Add certificates, runtimes and genres
            certificates.append(p1_dict['certificate'])
            runtimes.append(p1_dict['runtime'])
            genres.append(p1_dict['genre'])


            #p2 - Directors and Stars

                # Directors
            if('Director' in cleaned_string_p2 and 'Directors' not in cleaned_string_p2):
                start_index = cleaned_string_p2.index('Director:') + len('Director:')
                end_index = cleaned_string_p2.index('Stars:')
                result_string = cleaned_string_p2[start_index:end_index].strip()
                result_string = result_string.replace('\n', '')
                directors.append(result_string)


            if('Directors' in cleaned_string_p2):
                start_index = cleaned_string_p2.index('Directors:') + len('Directors:')
                end_index = cleaned_string_p2.index('Stars:')
                result_string = cleaned_string_p2[start_index:end_index].strip()
                result_string = result_string.replace('\n', '')
                directors.append(result_string)

            # Stars    
            start_index = cleaned_string_p2.index('Stars:') + len('Stars:')
            result_string = cleaned_string_p2[start_index:].strip()
            result_string = result_string.replace('\n', '')
            actors.append(result_string)
            
            # Scrape items inside each movie
            movie_url = f"https://www.imdb.com/title/{movie_id}/?ref_=ttls_li_tt"
            movie_response = requests.get(movie_url, headers=headers)
            movie_soup = BeautifulSoup(movie_response.content, "html.parser")

            #Get Publishers
            try:
                publisher_ul = movie_soup.find_all("ul", {"class": "ipc-inline-list ipc-inline-list--show-dividers ipc-inline-list--inline ipc-metadata-list-item__list-content base"})[9]
                publisher_li = publisher_ul.find_all("li")

                a_tags = []

                for li in publisher_li:
                    try:
                        a_tag = li.find('a')
                        a_tags.append(a_tag.text)
                    except AttributeError:
                        a_tags.append('Unknown Publisher')

                publishers.append(a_tags)
            except IndexError:
                publishers.append(['Unknown Publisher'])

            #Scrape reviews
            review_url = f"https://www.imdb.com/title/{movie_id}/reviews?ref_=tt_urv"
            review_response = requests.get(review_url, headers=headers)
            review_soup = BeautifulSoup(review_response.content, "html.parser")

            review_divs = review_soup.find_all("div", {"class": "text show-more__control"})

            movie_reviews = []

            for review in review_divs:
                movie_reviews.append(review.text)

            reviews.append(movie_reviews)

            #Scrape review stars and users
            review_container = review_soup.find_all("div", {"class": "lister-item-content"})
            review_star_divs = []
            review_user_text = []
            #review_star_divs = review_soup.find_all("span", {"class": "rating-other-user-rating"})

            for div in review_container:
                #Get stars
                stars = div.find("span", {"class": "rating-other-user-rating"})
                review_star_divs.append(stars)
                
                users = div.find_all("a")[1].text
                review_user_text.append(users)
                
                #review_user_divs.append(users)
                

            movie_review_stars = []
            movie_review_users = []

            for star in review_star_divs:
                try:
                    new_string = star.text.replace('\n', '')
                    movie_review_stars.append(new_string)
                except AttributeError:
                    movie_review_stars.append("No rating")
                    
            for user in review_user_text:
                movie_review_users.append(user)

            review_score.append(movie_review_stars)
            
            review_users.append(movie_review_users)
            
            #Page
            imdb_page.append(page)

            count += 1

            print(f"Movie {count} complete!")
    
    data = {'id': movie_ids, 'title': movie_titles, 'release year': release_years, "certificate": certificates, "runtime": runtimes, "genre": genres, "rating": ratings, "summary": summaries, "directors": directors, "actors": actors, "lifetime gross in $": lifetime_grosses, "reviews": reviews, "review_score": review_score, "review_user": review_users, "publishers": publishers, "page": imdb_page}
    df = pd.DataFrame(data)
    return df

In [3]:
# Save to CSV
get_movies().to_csv('movies_new.csv', index=False)

Movie 1 complete!
Movie 2 complete!
Movie 3 complete!
Movie 4 complete!
Movie 5 complete!
Movie 6 complete!
Movie 7 complete!
Movie 8 complete!
Movie 9 complete!
Movie 10 complete!
Movie 11 complete!
Movie 12 complete!
Movie 13 complete!
Movie 14 complete!
Movie 15 complete!
Movie 16 complete!
Movie 17 complete!
Movie 18 complete!
Movie 19 complete!
Movie 20 complete!
Movie 21 complete!
Movie 22 complete!
Movie 23 complete!
Movie 24 complete!
Movie 25 complete!
Movie 26 complete!
Movie 27 complete!
Movie 28 complete!
Movie 29 complete!
Movie 30 complete!
Movie 31 complete!
Movie 32 complete!
Movie 33 complete!
Movie 34 complete!
Movie 35 complete!
Movie 36 complete!
Movie 37 complete!
Movie 38 complete!
Movie 39 complete!
Movie 40 complete!
Movie 41 complete!
Movie 42 complete!
Movie 43 complete!
Movie 44 complete!
Movie 45 complete!
Movie 46 complete!
Movie 47 complete!
Movie 48 complete!
Movie 49 complete!
Movie 50 complete!
Movie 51 complete!
Movie 52 complete!
Movie 53 complete!
Mo

Movie 417 complete!
Movie 418 complete!
Movie 419 complete!
Movie 420 complete!
Movie 421 complete!
Movie 422 complete!
Movie 423 complete!
Movie 424 complete!
Movie 425 complete!
Movie 426 complete!
Movie 427 complete!
Movie 428 complete!
Movie 429 complete!
Movie 430 complete!
Movie 431 complete!
Movie 432 complete!
Movie 433 complete!
Movie 434 complete!
Movie 435 complete!
Movie 436 complete!
Movie 437 complete!
Movie 438 complete!
Movie 439 complete!
Movie 440 complete!
Movie 441 complete!
Movie 442 complete!
Movie 443 complete!
Movie 444 complete!
Movie 445 complete!
Movie 446 complete!
Movie 447 complete!
Movie 448 complete!
Movie 449 complete!
Movie 450 complete!
Movie 451 complete!
Movie 452 complete!
Movie 453 complete!
Movie 454 complete!
Movie 455 complete!
Movie 456 complete!
Movie 457 complete!
Movie 458 complete!
Movie 459 complete!
Movie 460 complete!
Movie 461 complete!
Movie 462 complete!
Movie 463 complete!
Movie 464 complete!
Movie 465 complete!
Movie 466 complete!


Movie 827 complete!
Movie 828 complete!
Movie 829 complete!
Movie 830 complete!
Movie 831 complete!
Movie 832 complete!
Movie 833 complete!
Movie 834 complete!
Movie 835 complete!
Movie 836 complete!
Movie 837 complete!
Movie 838 complete!
Movie 839 complete!
Movie 840 complete!
Movie 841 complete!
Movie 842 complete!
Movie 843 complete!
Movie 844 complete!
Movie 845 complete!
Movie 846 complete!
Movie 847 complete!
Movie 848 complete!
Movie 849 complete!
Movie 850 complete!
Movie 851 complete!
Movie 852 complete!
Movie 853 complete!
Movie 854 complete!
Movie 855 complete!
Movie 856 complete!
Movie 857 complete!
Movie 858 complete!
Movie 859 complete!
Movie 860 complete!
Movie 861 complete!
Movie 862 complete!
Movie 863 complete!
Movie 864 complete!
Movie 865 complete!
Movie 866 complete!
Movie 867 complete!
Movie 868 complete!
Movie 869 complete!
Movie 870 complete!
Movie 871 complete!
Movie 872 complete!
Movie 873 complete!
Movie 874 complete!
Movie 875 complete!
Movie 876 complete!
